In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class GEDA:
    def __init__(self, init_state, alpha=0.5, k=2):
        self.state = init_state
        # init params
        self.mu = init_state.mean(axis = 0)
        self.n = init_state.shape[0]
        self.m = init_state.shape[1]
        self.sigma = np.sqrt(((init_state - init_state.mean(axis = 0)) ** 2).sum(axis = 0) / self.n)
        self.alpha = alpha
        self.topk = k
    
    def loop(self, func):
        # choose the k best fitness indexes
        fit = np.apply_along_axis(func, 1, self.state)
        # round to 0.01 
        fit = np.apply_along_axis(lambda x: np.round(x, 2), 0, fit)
        fit_sorted = sorted(range(len(fit)), key=lambda i: fit[i])
        indexes = fit_sorted[-self.topk:]
        # update params
        self.mu = (1 - self.alpha) * self.mu + \
                   self.alpha * (self.state[fit_sorted[-1]] + self.state[fit_sorted[-2]] - self.state[fit_sorted[0]])
        topk_state = self.state[indexes]
        self.sigma = (1 - self.alpha) * self.sigma + \
                   self.alpha * np.sqrt(((topk_state - topk_state.mean(axis = 0)) ** 2).sum(axis = 0) / self.topk)
        # round to 0.1
        self.mu = np.apply_along_axis(lambda x: np.round(x, 1), 0, self.mu)
        self.sigma = np.apply_along_axis(lambda x: np.round(x, 1), 0, self.sigma)
        # resample 
        self.state = np.zeros_like(self.state)
        for i in range(self.m):
            self.state[:, i] = np.random.normal(self.mu[i], self.sigma[i], self.n)
            self.state[:, i] = np.apply_along_axis(lambda x: np.round(x, 1), 0, self.state[:, i])
        self.state = np.clip(self.state, -5, 5)

In [3]:
def func(*args):
    return round(1 / ((np.array(args) ** 2).sum() + 1), 2)

In [11]:
if __name__ == "__main__":
    init_state = np.array([[-2.1, 4.1],
              [1.0, -2.7],
              [2.2, -0.2],
              [-3.1, 1.1],
             ])
    num = 0
    for i in range(1000):
        g = GEDA(init_state, alpha=0.3)
        for j in range(1000):
            g.loop(func)
        if np.max(np.apply_along_axis(func, 1, g.state)) >= 0.99:
            num += 1 
    print(f"{num / 10} %")

85.5 %
